## Imports

In [173]:
import requests
import bs4
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, Float, String, DateTime, Date
from sqlalchemy.schema import CreateTable
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from datetime import date

## Part 1: Data Preprocessing

### Uber Sample Data Cleaning

In [9]:
# load_data
uber_sample = pd.read_csv("data/uber_rides_sample.csv")

In [10]:
# constraint on longitude and latitude
uber_sample = uber_sample[uber_sample['pickup_longitude']>=-74.242330]
uber_sample = uber_sample[uber_sample['pickup_longitude']<=-73.717047]
uber_sample = uber_sample[uber_sample['pickup_latitude']>=40.560445]
uber_sample = uber_sample[uber_sample['pickup_latitude']<=40.908524]
uber_sample = uber_sample[uber_sample['dropoff_longitude']>=-74.242330]
uber_sample = uber_sample[uber_sample['dropoff_longitude']<=-73.717047]
uber_sample = uber_sample[uber_sample['dropoff_latitude']>=40.560445]
uber_sample = uber_sample[uber_sample['dropoff_latitude']<=40.908524]

In [12]:
# function for calculating distance given longitude and latitude of pickup and dropoff location
def cal_distance(lat1, lat2, long1, long2):
     
    # convert degrees to radians
    long1 = radians(long1)
    long2 = radians(long2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # apply the Haversine formula
    result = sin((lat2 - lat1) / 2)**2 + cos(lat1) * cos(lat2) * sin((long2 - long1) / 2)**2
    result = 2 * asin(sqrt(result))*3956
      
    return result

In [88]:
# unit test for cal_distance
def cal_distance_test():
    assert round(cal_distance(40, 41, 70, 71), 2) == 86.74
cal_distance_test()

In [13]:
# filter out unnecessary columns
uber_cols = ['pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
uber_sample_cleaned = uber_sample[uber_cols].reset_index(drop=True)

In [14]:
# calculate distance for each trip and add it to the dataframe
distance = []
for i in range(len(uber_sample_cleaned)):
    data = uber_sample_cleaned.iloc[i,:]
    distance.append(cal_distance(data[2], data[4], data[1], data[3]))
uber_sample_cleaned['distance'] = distance

In [15]:
# save cleaned data to csv
uber_sample_cleaned.to_csv("data/uber_sample_cleaned.csv")

### count number of records in each month

In [16]:
uber_sample_cleaned['year'] = uber_sample_cleaned['pickup_datetime'].apply(lambda x:int(x[:4]))
uber_sample_cleaned['month'] = uber_sample_cleaned['pickup_datetime'].apply(lambda x:int(x[5:7]))

In [17]:
nrows_dict = uber_sample_cleaned.groupby(['year', 'month']).agg({'pickup_datetime':'count'}).to_dict()['pickup_datetime']

In [78]:
uber_sample_cleaned.drop(columns=['year', 'month'], inplace=True)

### Yellow Taxi Data Cleaning

#### Download and Sampling Yellow Taxi Data

In [11]:
# indicator function to check whether the time of the trip is within our desired time range
def isValidTime(year, month):
    if year > 2015 or year < 2009:
        return False
    if year == 2015 and month > 6:
        return False
    return True

In [12]:
# function to automatically scrap all the yellow taxi data within our desired time range
# the input parameter last_month is used to continue on unfinished downloading
def download_yellow_taxi(startpoint=None):
    # use requests to get the html of the yellow taxi page
    response = requests.get("https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page")

    # use beautifulsoup to parse the html
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    
    if startpoint:
        start = False
    else:
        start = True
    # extract all the urls that point to the csv files of yellow taxi trip records
    for row in soup.find_all(title="Yellow Taxi Trip Records"):
        url = row['href']
        # check if the csv is within our desired time range
        year = int(url[-11:-7])
        month = int(url[-6:-4])
        if (not start) and ((year, month) == startpoint):
            start = True
        if isValidTime(year, month) and start:
            # download the original csv files
            print(url)
            data = requests.get(url)
            fpath = "data/yellow_taxi/yt_{}_{}.csv".format(year, month)
            with open(fpath, 'wb')as file:
                file.write(data.content)
            print("Downloaded Successfully")
        
            temp = pd.read_csv(fpath, error_bad_lines=False)
            temp = temp.sample(nrows_dict[(year, month)]).reset_index(drop=True)
            
            temp.to_csv(fpath)
            print("Written Successfully")

In [18]:
download_yellow_taxi((2010, 4))

https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-04.csv
Downloaded Successfully
Written Successfully
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-05.csv
Downloaded Successfully
Written Successfully
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-06.csv
Downloaded Successfully
Written Successfully
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-07.csv
Downloaded Successfully
Written Successfully
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-08.csv
Downloaded Successfully
Written Successfully
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-09.csv
Downloaded Successfully
Written Successfully
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-10.csv
Downloaded Successfully
Written Successfully
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-11.csv
Downloaded Successfully
Written Successfully
https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2010-12.csv
D

In [66]:
# There three different column namings among all the csv
yt_cols_1 = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 
             'dropoff_longitude', 'dropoff_latitude', 'tip_amount']
yt_cols_2 = ['Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Start_Lon', 'Start_Lat',
             'End_Lon', 'End_Lat', 'Tip_Amt']
yt_cols_3 = ['pickup_datetime', 'dropoff_datetime', 'pickup_longitude', 'pickup_latitude',
             'dropoff_longitude', 'dropoff_latitude', 'tip_amount']

In [147]:
def clean_yellow_taxi():
    yt_sample = pd.DataFrame()
    for key in nrows_dict:
        # read in all the yellow taxi csv
        fpath = "data/yellow_taxi/yt_{}_{}.csv".format(key[0], key[1])
        temp = pd.read_csv(fpath)
        
        # strip the column names as some of them contain leading spaces
        temp.columns = [i.strip() for i in temp.columns]
        
        # select only necessary columns
        if 'tpep_pickup_datetime' in temp.columns:
            temp = temp[yt_cols_1]
        elif 'Trip_Pickup_DateTime' in temp.columns:
            temp = temp[yt_cols_2]
        else:
            temp = temp[yt_cols_3]
            
        # set the column names to be the same
        temp.columns = yt_cols_3
        
        # filter out invalid latitude and longitude
        temp = temp[temp['pickup_longitude']>=-74.242330]
        temp = temp[temp['pickup_longitude']<=-73.717047]
        temp = temp[temp['pickup_latitude']>=40.560445]
        temp = temp[temp['pickup_latitude']<=40.908524]
        temp = temp[temp['dropoff_longitude']>=-74.242330]
        temp = temp[temp['dropoff_longitude']<=-73.717047]
        temp = temp[temp['dropoff_latitude']>=40.560445]
        temp = temp[temp['dropoff_latitude']<=40.908524]
        
        # calculate distance
        distance = []
        for i in range(len(temp)):
            data = temp.iloc[i,:]
            distance.append(cal_distance(data[3], data[5], data[2], data[4]))
        temp['distance'] = distance
        
        # combine the dataframe from each month together 
        yt_sample = pd.concat([yt_sample, temp])
        
    return yt_sample.reset_index(drop=True)

In [148]:
yt_sample_cleaned = clean_yellow_taxi()

In [150]:
yt_sample_cleaned.to_csv("data/yt_sample_cleaned.csv")

### Weather Data

In [104]:
# function to clean up precipitation record
def transform_precipitation(p):
    if p==0.0:
        return p
    elif p=='T':
        p = 0.0
    else:
        if 's' in p:
            p = p[:-1]
        p = float(p)
    return p

In [108]:
# function to check whether the date is in our desired time range
def valid_date(date):
    date = pd.to_datetime(date)
    if date.month>6:
        return False
    else:
        return True

In [148]:
# function to extract daily weather information
def extract_daily(year):        
    df = pd.read_csv("data/{}_weather.csv".format(year), infer_datetime_format=True)
    daily_cols = ['DATE', 'DailyPrecipitation', 'DailyAverageWindSpeed', 'DailySustainedWindSpeed'] # necessary columns
    df = df[daily_cols]
    
    # filter out data after 2015-06
    if year == 2015:
        df['keep'] = df['DATE'].apply(lambda x:valid_date(x))
        df = df[df['keep']==True]
        df.drop(columns=['keep'], inplace=True)

    df.DATE = df.DATE.apply(lambda x:x[:10])
    df = df.fillna(0.0)
    df.DailyPrecipitation = df.DailyPrecipitation.apply(lambda x:transform_precipitation(x))
    df = df.groupby('DATE').agg('sum').reset_index()
        
    return df

In [123]:
# function to extract hourly weather information
def extract_hourly(year):
    hourly_cols = ['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed'] # necessary columns
    df = pd.read_csv("data/{}_weather.csv".format(year), infer_datetime_format=True)
    df = df[hourly_cols]
    
    # filter out data after 2015-06
    if year == 2015:
        df['keep'] = df['DATE'].apply(lambda x:valid_date(x))
        df = df[df['keep']==True]
        df.drop(columns=['keep'], inplace=True)
    
    # fill in NA values with 0.0 and clean up precipitation record
    df.fillna(0.0, inplace=True)
    df.HourlyPrecipitation = df.HourlyPrecipitation.apply(lambda x:transform_precipitation(x))
    
    return df

In [124]:
def clean_weather():
    weather_hourly = pd.DataFrame() # place holder for the hourly data
    weather_daily = pd.DataFrame() # place holder for the daily data
    
    # read in csv of each month and extract hourly and daily information, then combine them into the final result
    for year in range(2009, 2016):
        temp_hourly = extract_hourly(year)
        temp_daily = extract_daily(year)
        weather_hourly = pd.concat([weather_hourly, temp_hourly])
        weather_daily = pd.concat([weather_daily, temp_daily])
    weather_hourly.reset_index(drop=True, inplace=True)
    weather_daily.reset_index(drop=True, inplace=True)

    return weather_hourly, weather_daily

In [151]:
weather_hourly, weather_daily = clean_weather()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (8,9,10,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (7,8,9,10,17,18,42,65) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-pac

In [156]:
weather_hourly.to_csv("data/weather_hourly.csv")

In [155]:
weather_daily.to_csv("data/weather_daily.csv")

## Part 2: Storing Data

### Optional Data Loading

In [8]:
# used to load cleaned data from previous part

In [157]:
uber_sample_cleaned = pd.read_csv("data/uber_sample_cleaned.csv", index_col=0)
yt_sample_cleaned = pd.read_csv("data/yt_sample_cleaned.csv", index_col=0)
weather_hourly = pd.read_csv("data/weather_hourly.csv", index_col=0)
weather_daily = pd.read_csv("data/weather_daily.csv", index_col=0)

### Declare and Create Tables

In [183]:
engine = create_engine(f"sqlite:///project.db", echo=True)
Base = declarative_base()

In [184]:
# declare uber table
class Uber(Base):
    # YOUR CODE HERE
    __tablename__ = "uber"
    
    id = Column(Integer, primary_key=True)
    pickup_datetime = Column(DateTime)
    pickup_longitude = Column(Float)
    pickup_latitude = Column(Float)
    dropoff_longitude = Column(Float)
    dropoff_latitude = Column(Float)
    distance = Column(Float)

In [185]:
# declare yellow taxi table
class YellowTaxi(Base):
    # YOUR CODE HERE
    __tablename__ = "yellow_taxi"
    
    id = Column(Integer, primary_key=True)
    pickup_datetime = Column(DateTime)
    dropoff_datetime = Column(DateTime)
    pickup_longitude = Column(Float)
    pickup_latitude = Column(Float)
    dropoff_longitude = Column(Float)
    dropoff_latitude = Column(Float)
    tip_amount = Column(Float)
    distance = Column(Float)

In [186]:
# declare hourly weather table
class WeatherHourly(Base):
    __tablename__ = "weather_hourly"
    
    id = Column(Integer, primary_key=True)
    DATE = Column(DateTime)
    HourlyPrecipitation = Column(Float)
    HourlyWindSpeed = Column(Float)

In [187]:
# declare daily weather table
class WeatherDaily(Base):
    __tablename__ = "weather_daily"
    
    id = Column(Integer, primary_key=True)
    DATE = Column(Date)
    DailyPrecipitation = Column(Float)
    DailyAverageWindSpeed = Column(Float)
    DailySustainedWindSpeed = Column(Float)

In [188]:
Base.metadata.create_all(engine)

2022-04-21 22:54:47,829 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 22:54:47,830 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("uber")
2022-04-21 22:54:47,831 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 22:54:47,832 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("uber")
2022-04-21 22:54:47,833 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 22:54:47,835 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("yellow_taxi")
2022-04-21 22:54:47,835 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 22:54:47,836 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("yellow_taxi")
2022-04-21 22:54:47,837 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 22:54:47,838 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("weather_hourly")
2022-04-21 22:54:47,839 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 22:54:47,840 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("weather_hourly")
2022-04-21 22:54:47,841 INFO sqlalchemy.engine.Eng

### Populate Tables

In [189]:
uber_sample_cleaned.to_sql('uber', con=engine, index=True, index_label='id', if_exists='append')

2022-04-21 22:54:58,931 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("uber")
2022-04-21 22:54:58,932 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 22:54:59,085 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 22:55:00,171 INFO sqlalchemy.engine.Engine INSERT INTO uber (id, pickup_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, distance) VALUES (?, ?, ?, ?, ?, ?, ?)
2022-04-21 22:55:00,171 INFO sqlalchemy.engine.Engine [generated in 0.87812s] ((0, '2015-05-07 19:52:06 UTC', -73.99981689453125, 40.73835372924805, -73.99951171875, 40.72321701049805, 1.0452401206256468), (1, '2009-07-17 20:04:56 UTC', -73.994355, 40.728225, -73.99471, 40.750325, 1.5260124910108233), (2, '2009-08-24 21:45:00 UTC', -74.005043, 40.74077, -73.962565, 40.772647, 3.127281143399981), (3, '2009-06-26 08:22:21 UTC', -73.976124, 40.790844, -73.965316, 40.803349, 1.0318034471150883), (4, '2014-08-28 17:47:00 UTC', -73.925023, 40.744085, -73.97308199999998, 40

In [190]:
yt_sample_cleaned.to_sql('yellow_taxi', con=engine, index=True, index_label='id', if_exists='append')

2022-04-21 22:55:25,516 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("yellow_taxi")
2022-04-21 22:55:25,517 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 22:55:25,643 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 22:55:26,848 INFO sqlalchemy.engine.Engine INSERT INTO yellow_taxi (id, pickup_datetime, dropoff_datetime, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, tip_amount, distance) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-04-21 22:55:26,848 INFO sqlalchemy.engine.Engine [generated in 1.01214s] ((0, '2009-01-27 18:04:00', '2009-01-27 18:06:00', -73.982228, 40.767275, -73.975392, 40.764442, 1.3, 0.40749657390756977), (1, '2009-01-20 12:36:00', '2009-01-20 12:52:00', -73.966172, 40.753807, -74.000485, 40.737608, 0.0, 2.114858592965408), (2, '2009-01-17 02:02:05', '2009-01-17 02:10:04', -73.999893, 40.726838, -73.98330799999998, 40.722851, 0.0, 0.9104421427634034), (3, '2009-01-07 01:01:24', '2009-01-07 01:04:58', -73.97466799999998, 

In [191]:
weather_hourly.to_sql('weather_hourly', con=engine, index=True, index_label='id', if_exists='append')

2022-04-21 22:56:06,296 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("weather_hourly")
2022-04-21 22:56:06,297 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 22:56:06,321 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 22:56:06,625 INFO sqlalchemy.engine.Engine INSERT INTO weather_hourly (id, "DATE", "HourlyPrecipitation", "HourlyWindSpeed") VALUES (?, ?, ?, ?)
2022-04-21 22:56:06,625 INFO sqlalchemy.engine.Engine [generated in 0.24151s] ((0, '2009-01-01T00:51:00', 0.0, 18.0), (1, '2009-01-01T01:51:00', 0.0, 18.0), (2, '2009-01-01T02:51:00', 0.0, 18.0), (3, '2009-01-01T03:51:00', 0.0, 8.0), (4, '2009-01-01T04:51:00', 0.0, 11.0), (5, '2009-01-01T05:51:00', 0.0, 18.0), (6, '2009-01-01T06:51:00', 0.0, 14.0), (7, '2009-01-01T07:51:00', 0.0, 8.0)  ... displaying 10 of 72411 total bound parameter sets ...  (72409, '2015-06-30T23:59:00', 0.0, 0.0), (72410, '2015-06-30T23:59:00', 0.0, 0.0))
2022-04-21 22:56:06,719 INFO sqlalchemy.engine.Engine COMMIT


In [192]:
weather_daily.to_sql('weather_daily', con=engine, index=True, index_label='id', if_exists='append')

2022-04-21 22:56:16,233 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("weather_daily")
2022-04-21 22:56:16,234 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 22:56:16,239 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 22:56:16,253 INFO sqlalchemy.engine.Engine INSERT INTO weather_daily (id, "DATE", "DailyPrecipitation", "DailyAverageWindSpeed", "DailySustainedWindSpeed") VALUES (?, ?, ?, ?, ?)
2022-04-21 22:56:16,255 INFO sqlalchemy.engine.Engine [generated in 0.01159s] ((0, '2009-01-01', 0.0, 0.0, 0.0), (1, '2009-01-02', 0.0, 0.0, 0.0), (2, '2009-01-03', 0.0, 0.0, 0.0), (3, '2009-01-04', 0.0, 0.0, 0.0), (4, '2009-01-05', 0.0, 0.0, 0.0), (5, '2009-01-06', 0.0, 0.0, 0.0), (6, '2009-01-07', 0.0, 0.0, 0.0), (7, '2009-01-08', 0.0, 0.0, 0.0)  ... displaying 10 of 2367 total bound parameter sets ...  (2365, '2015-06-29', 0.0, 4.8, 12.0), (2366, '2015-06-30', 0.04, 4.9, 14.0))
2022-04-21 22:56:16,263 INFO sqlalchemy.engine.Engine COMMIT


### Generate Schema.sql

In [208]:
print(CreateTable(Uber.__table__).compile(engine))


CREATE TABLE uber (
	id INTEGER NOT NULL, 
	pickup_datetime DATETIME, 
	pickup_longitude FLOAT, 
	pickup_latitude FLOAT, 
	dropoff_longitude FLOAT, 
	dropoff_latitude FLOAT, 
	distance FLOAT, 
	PRIMARY KEY (id)
)




In [201]:
print(CreateTable(YellowTaxi.__table__).compile(engine))


CREATE TABLE yellow_taxi (
	id INTEGER NOT NULL, 
	pickup_datetime DATETIME, 
	dropoff_datetime DATETIME, 
	pickup_longitude FLOAT, 
	pickup_latitude FLOAT, 
	dropoff_longitude FLOAT, 
	dropoff_latitude FLOAT, 
	tip_amount FLOAT, 
	distance FLOAT, 
	PRIMARY KEY (id)
)




In [203]:
print(CreateTable(WeatherHourly.__table__).compile(engine))


CREATE TABLE weather_hourly (
	id INTEGER NOT NULL, 
	"DATE" DATETIME, 
	"HourlyPrecipitation" FLOAT, 
	"HourlyWindSpeed" FLOAT, 
	PRIMARY KEY (id)
)




In [202]:
print(CreateTable(WeatherDaily.__table__).compile(engine))


CREATE TABLE weather_daily (
	id INTEGER NOT NULL, 
	"DATE" DATE, 
	"DailyPrecipitation" FLOAT, 
	"DailyAverageWindSpeed" FLOAT, 
	"DailySustainedWindSpeed" FLOAT, 
	PRIMARY KEY (id)
)


